In [1]:
import fileinput
import string
from collections import defaultdict, Counter
from collections.abc import Callable
from itertools import filterfalse
import random

https://www.nytimes.com/games/wordle/index.html

In [2]:
def inc_exc(words: list[str], 
            letter_groups: dict[str, set[str]], 
            include: str, 
            exclude: str) -> list[str]:
    """Filters words based on the characters in include and exclude.
    
    If include and exclude are empty, inc_exc returns the same 
    list of words, albeit a copy of the original list with duplicate words removed.
    
    Args:
        words: A list of words.
        letter_groups: A mapping between a character in a word and 
            a subset from words that contain that character.
        include: A string of characters that must be contained in all the output words.
        exclude: A string of characters that must _not_ be contained in any output words.
        
    Returns:
        A list of filtered words where each word contains all the characters in include, but 
        does not have the characters listed in exclude.
    """
    include, exclude = set(include), set(exclude)
    possible = set(words)
    
    for ch in include:
        possible &= letter_groups[ch]

    print(f'Words that include {include}: {len(possible):,}')
    for ch in exclude:
        possible -= letter_groups[ch]

    print(f'Words that include {include}, but exclude {exclude}: {len(possible):,}')
    
    return list(possible)

def group_by_letter(words: list[str], char_set: str=string.ascii_lowercase) -> dict[str, set[str]]:
    """Groups words into sets based on what character is contained in the word.
    
    Args:
        words: A list of words.
        char_set: A string containing all the characters used to compose words.
        
    Returns:
        A mapping between each character in char_set and 
        a subset of words that contain that character.
    """
    letters = defaultdict(set)
    for ch in char_set:
        for word in words:
            if ch in word:
                letters[ch].add(word)
    return letters

class Possible(object):
    def __init__(self, iterable: list[str]):
        print(f'Sorting values')
        iterable.sort()
        self.values = iterable
    
    def distinct_letters_only(self):
        self.filter(lambda word: len(set(word)) == len(word))
        
    def filter(self, condition: Callable[[str], bool]):
        self.values = list(filter(condition, self.values))
        
    def __repr__(self):
        return f'Possible({self.values})'
    
    def __len__(self):
        return len(self.values)
        

In [3]:
words = [word.strip() for word in fileinput.input('words_alpha.txt')]
print(f'All words: {len(words):,}')

five_letter_words = [word for word in words if len(word) == 5]
five_letters_grouped = group_by_letter(five_letter_words)
five = lambda inc, exc: inc_exc(five_letter_words, five_letters_grouped, inc, exc)

print(f'Five-letter words: {len(five_letter_words):,}')

All words: 370,105
Five-letter words: 15,920


## Spelling Bee

In [4]:
grouped = group_by_letter(words)
all_words = lambda inc, exc: inc_exc(words, grouped, inc, exc)

In [5]:
excluded_letters = ''.join(sorted(set(string.ascii_lowercase) - set('turafhg')))
excluded_letters

'bcdeijklmnopqsvwxyz'

In [6]:
a = all_words('t', excluded_letters)
min_chars, max_chars = 4, 100
a = list(filter(lambda word: min_chars <= len(word) <= max_chars, a))

a.sort()
a.sort(key=len, reverse=True)

print(f'\nTotal number of possible words that are at least {min_chars} characters long, but no more than {max_chars} characters long: {len(a)}\n')
for word in a[:20]:
    print(f'{len(word):02}: {word}')

Words that include {'t'}: 182,889
Words that include {'t'}, but exclude {'k', 'j', 'd', 'n', 'x', 'y', 'm', 'z', 'e', 'c', 'p', 'o', 'w', 'v', 'b', 'l', 's', 'i', 'q'}: 165

Total number of possible words that are at least 4 characters long, but no more than 100 characters long: 117

09: arthragra
08: fatagaga
08: haftarah
07: fraught
07: ratatat
07: taharah
07: taratah
07: tathata
07: tautaug
07: thaught
07: tuatara
06: agatha
06: arguta
06: arthra
06: arthur
06: aurata
06: futtah
06: garrat
06: guttar
06: guttur


## Guesses

### Method 1: Uniformally random word

In [7]:
distinct_letters = list(filter(lambda word: len(set(word)) == len(word), five_letter_words))

In [8]:
random.choice(distinct_letters)

'quipo'

### Method 2: Pick words with letters that occur the most

The end goal is to eliminate as many words as possible.  For the first guess, if we pick a word that has letters that occur the most, we can see if any of those letters can be discarded.  Because we used the most frequently used letters, we theoretically eliminate more words per letter using these letters.

In [9]:
hist = Counter()
for word in five_letter_words:
    for ch in word:
        hist[ch] += 1

In [10]:
hist.most_common(10)

[('a', 8393),
 ('e', 7802),
 ('s', 6537),
 ('o', 5219),
 ('r', 5144),
 ('i', 5067),
 ('l', 4247),
 ('t', 4189),
 ('n', 4044),
 ('u', 3361)]

In [11]:
possible = five('aril', '')
possible[:10]

Words that include {'i', 'a', 'l', 'r'}: 46
Words that include {'i', 'a', 'l', 'r'}, but exclude set(): 46


['liard',
 'rials',
 'viral',
 'riyal',
 'rauli',
 'ranli',
 'pilar',
 'urial',
 'rival',
 'liars']

## Filter words

In [12]:
print(f'Words with five letters: {len(five_letter_words):,}')

Words with five letters: 15,920


In [15]:
words = Possible(five('ola', 'quipuenvymrf'))

words.filter(lambda word: word[3] != 'o')
words.filter(lambda word: word[1] != 'o' and word[2] != 'l' and word[3] == 'a')
words.filter(lambda word: word.endswith('loat'))
# words.distinct_letters_only()

print(len(words))
words

Words that include {'a', 'l', 'o'}: 301
Words that include {'a', 'l', 'o'}, but exclude {'y', 'm', 'p', 'n', 'u', 'e', 'f', 'r', 'i', 'v', 'q'}: 90
Sorting values
3


Possible(['bloat', 'gloat', 'sloat'])

## Save results

In [25]:
import sqlite3
from collections import namedtuple
import itertools

Result = namedtuple('Result', ['wordle', 'date', 'words', 'regular', 'high_contrast'])
def println(*objects, **kw):
    print(*objects, end='\n\n', **kw)
    
def print_results(results, show_rowid=False):
    def header(head, rowid):
        msg = f'{head}'
        if show_rowid:
            msg += f' rowid: {rowid}'
            
        return msg
        
    for rowid, *result in results:
        result = Result(*result)

        hi = result.high_contrast.splitlines()
        words = result.words.splitlines()
        
        print(header(hi[0], rowid))
        for line, word in itertools.zip_longest(itertools.islice(hi, 2, None), words, fillvalue=''):
            print(f'{line} {word}')
        print()
    
con = sqlite3.connect('wordle.db')
cur = con.cursor()

In [26]:
most_recent = cur.execute('''SELECT ROWID, * FROM results ORDER BY ROWID DESC''').fetchall()[:5]
print_results(most_recent)

Wordle 367 5/6*
⬜⬜⬜⬜⬜ quipu
⬜⬜⬜🟦⬜ envoy
⬜🟦🟦🟧⬜ molar
⬜🟧🟧🟧🟧 float
🟧🟧🟧🟧🟧 gloat

Wordle 366 3/6*
⬜⬜⬜🟧⬜ fraud
⬜⬜🟦🟧⬜ bonus
🟧🟧🟧🟧🟧 input

Wordle 357 3/6*
⬜⬜⬜⬜⬜ viral
🟦⬜🟧⬜🟧 smoke
🟧🟧🟧🟧🟧 goose

Wordle 350 5/6*
⬜⬜⬜⬜⬜ abyss
⬜🟦⬜⬜⬜ chunk
⬜⬜⬜🟧🟧 depth
🟧🟦🟦🟧🟧 forth
🟧🟧🟧🟧🟧 froth

Wordle 347 3/6*
⬜🟧⬜🟧⬜ trial
⬜🟧⬜🟧⬜ urban
🟧🟧🟧🟧🟧 creak



### Create table

Only need to do once.  Uncomment to execute.

In [18]:
# cur.execute('''
# CREATE TABLE results 
# (wordle, date, words, regular, high_contrast)
# ''')
# con.commit()

### Save result

To keep it simple, everything is stored as TEXT.

In [19]:
cur.execute('''
INSERT INTO results VALUES (
'367', '2022-06-21', 
'quipu
envoy
molar
float
gloat', 
'Wordle 367 5/6*

⬜⬜⬜⬜⬜
⬜⬜⬜🟨⬜
⬜🟨🟨🟩⬜
⬜🟩🟩🟩🟩
🟩🟩🟩🟩🟩', 
'Wordle 367 5/6*

⬜⬜⬜⬜⬜
⬜⬜⬜🟦⬜
⬜🟦🟦🟧⬜
⬜🟧🟧🟧🟧
🟧🟧🟧🟧🟧'
)''')

con.commit()

### SQL utilities

In [20]:
# cur.execute('''SELECT * FROM results WHERE wordle='325' ''').fetchall()

In [21]:
# cur.execute('''DELETE FROM results WHERE wordle='325' ''')
# con.commit()

In [22]:
# close connection to database
con.close()